In [1]:
import numpy as np 
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt


In [119]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import os

# Asignar la ruta donde deseas guardar las imágenes
PROJECT_ROOT_DIR = r"C:\Users\ezequ\OneDrive\Documentos\Facultad\Optimizacion\tp1"
CHAPTER_ID = "imagenes"  # El nombre de la capeta donde voy a guardar las imagenes

# Crear la ruta completa donde se guardarán las imágenes
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)



In [49]:
path = r'C:\Users\ezequ\OneDrive\Documentos\Facultad\Aprendizaje Automatico\material-main\tp\01_aprendizaje_supervisado\datos\data.csv'
df = pd.read_csv(path)


In [8]:
df['target'].value_counts()

target
0    343
1    157
Name: count, dtype: int64

La relacion entre clases es 0.45 siendo la mayoritaria la 0  por lo que estamos ante un dataset balanceado

A priori hacer un split 400 100 para desarrollo validacion suena razonable , estamos dejando 20% de nuestro dataset para validar.

Capaz es un poco mucho ese porcentaje pero creo que eso aseguraria que los resultados obtenidos tendran un poco mas de robustez.

Dentro de desarrollo capaz podemos hacer algo como k folds con k = 4 para sacarle un poco mas el jugo a los datos.

In [118]:
def train_test_split(df,cant_en_validacion = 100 ,proporcion = 0.45):
    res = []
    cant_positivas = proporcion * cant_en_validacion
    cant_negativas = cant_en_validacion - cant_positivas
    contador_positivos = 0
    contador_negativos = 0
    for index, value in enumerate(df['target']):
        if value == 1:
            contador_negativos += 1
            res.append(index)
        
        if contador_negativos == cant_negativas:
            break
    for index, value in enumerate(df['target']):

        if value == 0:
            contador_positivos += 1
            res.append(index)
        
        if contador_positivos == cant_positivas:
            break
        df_dev = df.loc[~df.index.isin(res)].reset_index(drop=True)
    #df_val = df.loc[res].reset_index(drop=True)
    #df_dev = df.loc[~df.index.isin(res)].reset_index(drop=True)
    return df_dev,df_val

In [50]:
df_dev , df_val = train_test_split(df)

In [51]:
df_dev

,v000,v001,v002,v003,v004,v005,v006,v007,v008,v009,...,v191,v192,v193,v194,v195,v196,v197,v198,v199,target
76,-0.624902,3.485426,-2.281974,2.417061,-11.210319,-1.521622,-16.872402,1.680587,-0.040097,2.058603,...,7.732963,-0.632881,-0.086670,-0.265007,-1.808472,-0.214784,-0.457782,0.387314,0.009503,0
79,-0.490494,-5.574436,-3.409070,-2.322581,6.719178,2.003021,-2.106789,0.031419,-0.836519,-0.539284,...,-8.025676,0.545177,0.673909,0.873688,0.662084,0.362046,0.299614,-0.083432,1.179663,0
80,0.750471,-0.315479,8.179536,0.798978,1.797776,0.687250,5.642598,0.004879,0.955422,-0.122200,...,-14.302386,0.675080,1.174305,-0.172391,0.593001,0.105022,-0.396634,1.043170,0.771605,0
81,0.887018,-1.211407,1.826868,3.994457,4.694885,-0.223417,4.001168,-0.412645,0.311031,-0.205814,...,2.026220,0.108079,0.169784,-1.207572,-0.035194,-0.032413,0.640339,0.489267,1.705443,0
85,0.284852,-0.323182,-2.530018,-1.979892,3.011604,0.171972,-5.489846,-1.540585,-0.675751,-0.949380,...,11.236575,-0.402194,0.319596,1.048651,-0.232624,-0.365793,1.390058,-0.292737,-0.838737,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-0.521610,3.695148,-4.472063,-3.860461,1.732738,0.082542,24.025503,0.313731,-0.547227,2.131908,...,-9.525284,1.785532,0.846785,1.047359,0.258732,0.660685,1.044754,0.556689,-1.315301,0
496,-1.003124,1.113742,4.296378,-1.997490,3.258672,-0.513932,3.677044,0.848930,-0.088891,-0.306494,...,16.937089,0.414220,-0.852210,-0.129207,-0.681531,-0.158394,0.816928,-0.204489,0.724944,0
497,1.084936,0.328886,4.545345,0.678987,0.003042,-0.807901,-13.653907,-0.654958,-0.860854,-1.017529,...,-20.574463,0.119288,0.505041,0.922128,0.405464,1.886947,0.407544,1.431193,0.159015,1
498,-0.808172,-2.116049,5.293979,5.019026,7.959400,0.717410,2.865135,0.837467,-1.083672,-0.275118,...,-5.208473,1.243521,-0.286814,-1.542903,-0.079130,1.120452,-1.559993,-1.969015,-0.735959,0


In [53]:
df_dev = df_dev.reset_index(drop=True)

df_train , df_test =  train_test_split(df_dev)


In [86]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, roc_auc_score , average_precision_score

In [56]:
X_train = df_train.drop(columns=['target']).values

y_train = df_train['target'].values

X_test  = df_test.drop(columns=['target']).values

y_test  = df_test['target'].values

X_val   = df_val.drop(columns=['target']).values

y_val   =  df_val['target'].values

In [59]:

tree_clf = DecisionTreeClassifier(max_depth=3, random_state=51)
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, random_state=51)

In [109]:
scoring = ['accuracy', 'roc_auc', 'average_precision']
skf = StratifiedKFold(n_splits=5)

cv_results_test = cross_validate(tree_clf, X_test, y_test, cv = skf, scoring = scoring)

print("Cross-validation results in Test:")
for metric in scoring:
    print(f"{metric.capitalize()}: {cv_results_test[f'test_{metric}'].round(3)}")
    print('\n')
    print(f"Promedio global de {metric.capitalize()}:  {cv_results_test[f'test_{metric}'].mean().round(3)}")

print("--------------------------------------------------------")

cv_results_val = cross_validate(tree_clf, X_val, y_val, cv = skf, scoring = scoring)

print("Cross-validation results in Validation:")
for metric in scoring:
    print(f"{metric.capitalize()}: {cv_results_val[f'test_{metric}'].round(3)}")
    print('\n')
    print(f"Promedio global de {metric.capitalize()}:  {cv_results_val[f'test_{metric}'].mean().round(3)}")

Cross-validation results in Test:
Accuracy: [0.55 0.35 0.65 0.7  0.45]


Promedio global de Accuracy:  0.54
Roc_auc: [0.52  0.328 0.591 0.601 0.364]


Promedio global de Roc_auc:  0.481
Average_precision: [0.56  0.486 0.63  0.762 0.493]


Promedio global de Average_precision:  0.586
--------------------------------------------------------
Cross-validation results in Validation:
Accuracy: [0.4  0.55 0.55 0.5  0.3 ]


Promedio global de Accuracy:  0.46
Roc_auc: [0.439 0.556 0.636 0.46  0.333]


Promedio global de Roc_auc:  0.485
Average_precision: [0.527 0.608 0.679 0.527 0.489]


Promedio global de Average_precision:  0.566


Basado en pagina 288 de Introduction to Machine Learning with Python

In [136]:
alturas = [3, 5, None]
criterios = ["gini", "entropy"]

resultados = {}

for altura in alturas:
    for criterio in criterios:
        
        tree = DecisionTreeClassifier(max_depth = altura, criterion = criterio, random_state = 51)
        
        tree.fit(X_train, y_train)
        
        cv_results_train = cross_validate(tree, X_train, y_train, cv = skf, scoring = 'accuracy')
        
        y_pred_val = tree.predict(X_val)

        accuracy_val = accuracy_score(y_val, y_pred_val)
        
        global_score_test= cv_results_train['test_score'].mean()
        
        resultados[(altura, criterio)] = (global_score_test,accuracy_val)

results_df = pd.DataFrame(resultados.values(), index=resultados.keys(), columns=['Test Score', 'Validation Score'])



In [137]:
results_df

Test Score  Validation Score
3   gini       0.826667              0.55
    entropy    0.803333              0.50
5   gini       0.780000              0.53
    entropy    0.786667              0.55
NaN gini       0.770000              0.52
    entropy    0.770000              0.58